In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
import os
import pathlib
import dotenv
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint


In [2]:
from dotenv import load_dotenv
load_dotenv()

openai_api_key     = os.getenv("OPENAI_API_KEY")
gemini_api_key     = os.getenv("GEMINI_API_KEY")
groq_api_key       = os.getenv("GROQ_API_KEY")
langchain_api_key  = os.getenv("LANGCHAIN_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

In [3]:
#loading The PDF

loader = PyPDFLoader(r"C:\Users\ishaa\Downloads\sample_medical_report.pdf")
medical_report = loader.load()
print("medical report loaded")
print(f"loaded doc has : {len(medical_report)} Page")


medical report loaded
loaded doc has : 1 Page


In [4]:
#text Spiltter 
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 20,
    length_function = len, #To count the CharS
)
chunks = splitter.split_documents(medical_report)
print("Chunks created")
print(f"{len(chunks)} : length of chunks")      

Chunks created
7 : length of chunks


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'} 
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print("Embedding model loaded From Huggingface")

C:\Users\ishaa\AppData\Local\Temp\ipykernel_19360\1603069342.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Embedding model loaded From Huggingface


In [6]:
#The embeddings and the vectpor Db part 
db = FAISS.from_documents(chunks,embeddings)
db.save_local("faiss_index")
current_directory = os.getcwd()
index_path = os.path.join(current_directory, "faiss_index")
print(f"The Embeddings have been made @ {index_path}")

The Embeddings have been made @ c:\Users\ishaa\OneDrive\Desktop\AgenticAi\Agentic-Ai\Udmy\Project-Synaptic\faiss_index


In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

print("Databade Loaded")

Databade Loaded


In [8]:
#Creating The context retriever 

retriever = db.as_retriever()
print(f"retriever created : \n {retriever}")

retriever created : 
 tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027D18E3F4D0> search_kwargs={}


In [9]:
# Lets make The user choose whatever model he/she wantes to use: 
#function will be Createsd to let  User Choose between 
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI
from langchain_perplexity import ChatPerplexity 
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage , HumanMessage

In [14]:
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
from langchain_perplexity import ChatPerplexity
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
from pprint import pprint
import os

In [15]:
def get_llm():

    while True:
        user_choice = input("Enter the model you want to use For the ChatBot Purposes (ollama, groq, perplexity): ").lower()

        if user_choice == "ollama":
            print("System : Starting Ollama model")
            return Ollama(model="llama3")
        
        elif user_choice == "groq":
            print("System : Starting model(qwen-32b)...")
            return ChatGroq(model="qwen/qwen3-32b", groq_api_key=os.getenv("GROQ_API_KEY"))
            
        elif user_choice == "perplexity":
            print("System : Starting model FROM PERPLEXITY")
            return ChatPerplexity(model="llama-3-sonar-large-32k-online", perplexity_api_key=os.getenv("PERPLEXITY_API_KEY"))
            
        else:
            print("INVALID CHOICE")

    print("System Started Successfully")

llm = get_llm()

System : Starting Ollama model


C:\Users\ishaa\AppData\Local\Temp\ipykernel_19360\814006130.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  return Ollama(model="llama3")


In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are an advanced AI medical assistant. Your primary goal is to answer questions using the provided medical report in the <context>. However, you may enrich your answers with general medical knowledge to provide clarity.

    **Your Core Operating Procedure:**
    1.  **Prioritize the Document:** Always start by looking for the answer in the provided <context>. This is your primary source of truth.
    2.  **Enrich with General Knowledge:** If the document mentions a specific medical term (e.g., "bradycardia"), first use the context to describe the patient's specific case. Then, you may add a general definition from your own knowledge.
    3.  **Attribute Your Sources (Crucial Rule):** You MUST clearly distinguish between information from the document and general knowledge. Use phrases like "According to the report..." or "For context, in general medicine...".
    4.  **Medical Disclaimer:** Never provide a direct diagnosis or medical advice. Always recommend consulting a qualified healthcare professional.
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", """
    **Context from the medical report:**
    <context>
    {context}
    </context>

    **User's Question:**
    {input}
    """),
])